## Part 1: Preprocessing

In [271]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from keras.models import Model
from keras import layers
from sklearn.preprocessing import OneHotEncoder


#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [272]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [273]:
# attrition_copy = attrition_df.copy()

# attrition_copy['Attrition'] = attrition_copy['Attrition'].map({'No': 0, 'Yes': 1})


# correlation = attrition_copy.corr()
# correlation['Attrition'].sort_values(ascending=False)

In [274]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [275]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'BusinessTravel', 'HourlyRate', 'DistanceFromHome', 'Education',
                     'MaritalStatus', 'JobSatisfaction', 'EnvironmentSatisfaction', 'YearsSinceLastPromotion']
                    


# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes


Age                         int64
BusinessTravel             object
HourlyRate                  int64
DistanceFromHome            int64
Education                   int64
MaritalStatus              object
JobSatisfaction             int64
EnvironmentSatisfaction     int64
YearsSinceLastPromotion     int64
dtype: object

In [276]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)


In [277]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [278]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [279]:
# Create a OneHotEncoder for the Department column
enc = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
enc.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded = enc.transform(y_train[['Department']])
y_test_encoded = enc.transform(y_test[['Department']])
y_train_encoded

c:\Users\deers\anaconda3\envs\dev\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [280]:
# Create a OneHotEncoder for the Attrition column
enc = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
enc.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded2 = enc.transform(y_train[['Attrition']])
y_test_encoded2 = enc.transform(y_test[['Attrition']])
y_train_encoded2


c:\Users\deers\anaconda3\envs\dev\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

## Create, Compile, and Train the Model

In [281]:
# Find the number of columns in the X training data
X_train.shape[1]

13

In [282]:
# Create the input layer
input_layer = layers.Input(shape=(X_train.shape[1],))

# Create at least two shared layers
shared_layer1 = layers.Dense(units=8, activation='relu', name='shared_layer1')(input_layer)
shared_layer2 = layers.Dense(units=8, activation='relu', name = 'shared_layer2')(shared_layer1)

In [283]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_branch = layers.Dense(units=8, activation='relu', name='department_hidden')(shared_layer2)


# Create the output layer
department_output = layers.Dense(units=3, activation='sigmoid', name='department_output')(department_branch)

In [284]:
# Create a branch for Attrition
# with a hidden layer and an output layer
# Create the hidden layer
attrition_hidden = layers.Dense(units=8, activation='relu', name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(units=2, activation='sigmoid', name='attrition_output')(attrition_hidden)

In [285]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'binary_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_41"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_17      │ (None, 13)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer1       │ (None, 8)         │        112 │ input_layer_17[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer2       │ (None, 8)         │         72 │ shared_layer1[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 8)         │         72 │ shared_layer2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 8)         │         72 │ shared_layer2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         27 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         18 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 373 (1.46 KB)

 Trainable params: 373 (1.46 KB)

 Non-trainable params: 0 (0.00 B)

In [286]:
# Train the model
model.fit(X_train_scaled,
          {'department_output': y_train_encoded, 'attrition_output': y_train_encoded2},
          epochs=100,
          shuffle=True,
          verbose=2)

Epoch 1/100


35/35 - 2s - 50ms/step - attrition_output_accuracy: 0.8230 - department_output_accuracy: 0.4419 - loss: 1.2770
Epoch 2/100
35/35 - 0s - 2ms/step - attrition_output_accuracy: 0.8276 - department_output_accuracy: 0.5544 - loss: 1.1804
Epoch 3/100
35/35 - 0s - 2ms/step - attrition_output_accuracy: 0.8276 - department_output_accuracy: 0.6180 - loss: 1.0944
Epoch 4/100
35/35 - 0s - 2ms/step - attrition_output_accuracy: 0.8276 - department_output_accuracy: 0.6543 - loss: 1.0277
Epoch 5/100
35/35 - 0s - 2ms/step - attrition_output_accuracy: 0.8276 - department_output_accuracy: 0.6642 - loss: 0.9810
Epoch 6/100
35/35 - 0s - 2ms/step - attrition_output_accuracy: 0.8276 - department_output_accuracy: 0.6661 - loss: 0.9506
Epoch 7/100
35/35 - 0s - 2ms/step - attrition_output_accuracy: 0.8276 - department_output_accuracy: 0.6661 - loss: 0.9323
Epoch 8/100
35/35 - 0s - 2ms/step - attrition_output_accuracy: 0.8276 - department_output_accuracy: 0.6661 - loss: 0.9229
Epoch 9/100
35/35 - 0s - 2ms/step -

In [288]:
# Evaluate the model with the testing data
attrition_accuracy, department_accuracy, total_loss = model.evaluate(
    X_test_scaled,
    {'department_output': y_test_encoded, 'attrition_output': y_test_encoded2},
    verbose=2
)

12/12 - 0s - 3ms/step - attrition_output_accuracy: 0.8723 - department_output_accuracy: 0.6114 - loss: 0.8953


In [290]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {department_accuracy}")
print(f"Attrition Accuracy: {attrition_accuracy}")

Department Accuracy: 0.8722826242446899
Attrition Accuracy: 0.8953461647033691


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 